# 신경망(Neural Network)

신경망은 `torch.nn` 패키지를 사용하여 생성할 수 있습니다.

지금까지 `autograd`를 살펴봤는데요, `nn`은 모델을 정의하고 미분하는데 `autograd`를 사용합니다. `nn.module`은 계층(layer)과 `output`을 반환하는 forward(input) 메서드를 포함하고 있습니다.

해당 신경망을 사용합니다.

![](https://tutorials.pytorch.kr/_images/mnist.png)

# 신경망 학습 과정

* 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의합니다.
* 데이터셋(dataset) 입력을 반복합니다.
* 입력을 신경망에서 전파(process) 합니다.
* 손실(loss: 출력이 정답으로부터 얼마나 떨어져 있는지)을 계산합니다.
* 변화도(gradient)를 신경망의 매개변수들에 역으로 전파합니다.
* 신경망의 가중치를 갱신합니다.
    * 가중치(weight) = 가충치(weight) - 학습률(learning Rate) * 변화도(Gradient)

# 신경망 정의

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)

        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s

        return num_features

In [2]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


`forward` 함수만 정의하고 나면, (변화도를 계산하는) `backward` 함수는 `autograd`를 사용하여 자동으로 정의됩니다. 

`forward` 에서는 어떠한 Tensor 연산을 사용해도 됩니다.

모델의 학습 가능한 매개변수들은 `net.parameters()`에 의해 반환됩니다.

In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


임의의 32 x 32 입력값을 넣어보겠습니다.

> 이 신경망(LeNet)의 예상되는 입력 크기는 32x32 입니다. 이 신경망에 MNIST 데이터셋을 사용하기 위해서는, 데이터셋의 이미지 크기를 32x32로 변경해야 합니다.

In [4]:
mnistFmt = torch.randn(1, 1, 32, 32)
out = net(mnistFmt)
print(out)

tensor([[-0.0728,  0.0637,  0.0667, -0.1568,  0.0263,  0.0531, -0.0582,  0.0173,
         -0.0895, -0.1106]], grad_fn=<AddmmBackward>)


모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고, 무작위 값으로 역전파를 합니다.

In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

`torch.nn`은 미치-배치(mini-batch)만 지원합니다. `torch.nn` 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니-배치만을 입력으로 받습니다.

예를 들어, `nnConv2D` 는 `nSamples x nChannels x Height x Width` 의 4차원 Tensor를 입력으로 합니다.

만약 하나의 샘플만 있다면, `mnistFmt.unsqueeze(0)`을 사용하여 가짜 차원을 추가합니다.

`torch.Tensor`
*  `backward()` 같은 autograd 연산을 지원하는 다차원 배열입니다. 또한 tensor에 대한 변화도(Gradient)를 갖고 있습니다.

`nn.Module`
* 신경망 모듈, 매개변수를 캡슐화(Encapsulation)하는 간편한 방법으로, GPU로 이동, 내보내기(exporting), 불러오기(Loading) 등의 작업을 위한 헬퍼(Helper)를 제공합니다.

`nn.Parameter`
* Tensor의 한 종류로, Module 에 속성으로 할당할 때 자동으로 매개변수로 등록됩니다.

`autograd.Function`
* autograd 연산의 전방향과 역방향 정의를 구현합니다.
* 모든 `Tensor` 연산은 하나 이사으이 `Function` 노드를 생성하며, 각 노드는 Tensor를 생성하고 이력(history)을 부호화하는 함수를 연결하고 있습니다.

# 손실 함수(Loss Fucntion)

손실 함수는 `(output, target)`을 한 쌍의 입력으로 받아, 출력(`output`)이 정답(`target`)으로부터 얼마나 멀리 떨어져있는지 추정하는 값을 계산합니다.

nn 패키지에는 여러가지 손실 함수들이 존재합니다. [Docs](https://pytorch.org/docs/stable/nn.html)


In [8]:
output = net(mnistFmt)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7654, grad_fn=<MseLossBackward>)


`.grad_fn` 속성을 사용하여 `loss` 를 역방향에서 따라가다보면, 다음과 같은 모습의 연산 그래프를 볼 수 있습니다.

In [ ]:
mnistFmt -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
         -> view -> linear -> relu -> linear -> relu -> linear
         -> MSELoss
         -> loss

따라서, `loss.backward()`를 실행할 때, 전체 그래프는 손실(loss)에 대하여 미분되며, 그래프 내의 `requires_grad=True` 인 모든 Tensor는 변화도(gradient)가 누적된 `.grad` Tensor를 갖게 됩니다.

In [9]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

# 역전파(Back propagation)

오차(error)를 역전파하기 위해서는 `loss.backward()` 만 해주면 됩니다. 기존 변화도를 없애는 작업이 필요하는데, 그렇지 않으면 변화도가 기존 변화도에 누적되기 때문입니다.

`loss.backward()`를 호출하여 역전파 전과 후에 conv1의 bias gradient를 살펴보겠습니다.

In [10]:
net.zero_grad()
print('conv1.bias.grad before backward: {}'.format(net.conv1.bias.grad))
loss.backward()
print('conv1.bias.grad after backward: {}'.format(net.conv1.bias.grad))

conv1.bias.grad before backward: tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward: tensor([ 0.0140,  0.0025,  0.0041, -0.0010,  0.0135, -0.0056])


# 가중치 갱신

확률적 경사하강법(SGD: Stohastic Gradient Descent)를 사용하여 가중치를 갱신합니다.

In [11]:
learningRate = 0.01
for f in net.parameters(): f.data.sub_(f.grad.data * learningRate)

신경망을 구성할 때 SGD, Nesterov-SGD, Adam, RMSProp 등과 같은 가중치 갱신 규칙을 `torch.optim` 내에 구현되어 있습니다.

In [13]:
import torch.optim as optim

# Create Optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# 학습 과정에서는 다음과 같이 구현합니다.
optimizer.zero_grad() 
output = net(mnistFmt)
loss = criterion(output, target)
loss.backward()
optimizer.step()

`optimizer.zero_grad()` 를 사용하여 수동으로 변화도 버퍼를 0으로 설정하여 변화도 누적을 방지합니다.